In [1]:
import torch
path_to_pth_file = "/content/mnist_cnn_model_simba.pth"
model = torch.load(path_to_pth_file)

import keras.datasets.mnist as mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [2]:
import numpy as np

conv1_wt=model['conv1.weight']
conv1_bias=model['conv1.bias']
fc1_wt=model['fc1.weight']
fc1_bias=model['fc1.bias']
fc2_wt=model['fc2.weight']
fc2_bias=model['fc2.bias']


conv1_wt=conv1_wt.numpy()
conv1_bias=conv1_bias.numpy()
fc1_wt=fc1_wt.numpy()
fc1_bias=fc1_bias.numpy()
fc2_wt=fc2_wt.numpy()
fc2_bias=fc2_bias.numpy()

test_images = test_images/255.0

In [3]:
def conv_2d(test_images, conv_wt, conv_bias):
    conv_outputs = []
    imgs = np.expand_dims(test_images, axis=1)
    batch_size, in_channels, in_height, in_width = imgs.shape
    out_channels, _, kernel_height, kernel_width = conv_wt.shape
    out_height = in_height - kernel_height + 1
    out_width = in_width - kernel_width + 1
    for img in imgs:
        img_conv_outputs = []
        for filter, b in zip(conv_wt, conv_bias):
            conv_output = np.zeros((out_height, out_width))
            for i in range(out_height):
                for j in range(out_width):
                    roi = img[:, i:i+kernel_height, j:j+kernel_width]
                    conv_output[i, j] = np.sum(roi * filter) + b
            img_conv_outputs.append(conv_output)

        conv_outputs.append(img_conv_outputs)

    conv_outputs = np.array(conv_outputs)
    return conv_outputs

In [9]:
conv_2d(test_images[0:1], conv1_wt, conv1_bias).shape

(1, 32, 26, 26)

In [4]:
def relu(input_data):
    return np.maximum(input_data, 0)

def fully_connected(input_data, weights, bias):
    return np.matmul(input_data, weights.T) + bias

def softmax(input_data):
    exp_values = np.exp(input_data - np.max(input_data, axis=1, keepdims=True))
    return exp_values / np.sum(exp_values, axis=1, keepdims=True)

In [5]:
def max_pool(conv_out, kernel_size):
    batch_size, channels, height, width = conv_out.shape
    out_height = height // kernel_size
    out_width = width // kernel_size

    output = np.zeros((batch_size, channels, out_height, out_width))

    for b in range(batch_size):
        for c in range(channels):
            for i in range(out_height):
                for j in range(out_width):
                    max_val = np.max(conv_out[b, c, kernel_size*i: kernel_size*(i+1), kernel_size*j: kernel_size*(j+1)])
                    output[b, c, i, j] = max_val
    return output

In [6]:

def forward_pass(input_data, conv1_wt, conv1_bias, fc1_wt, fc1_bias, fc2_wt, fc2_bias):

    print("input: ", input_data.shape)

    conv_output = conv_2d(input_data, conv1_wt, conv1_bias)
    print("convolution: ", conv_output.shape)

    conv_output = relu(conv_output)
    print("activation: ", conv_output.shape)

    pooled_output = max_pool(conv_output, kernel_size=2)
    print("max pooling: ", pooled_output.shape)

    flattened_output = pooled_output.reshape(len(input_data), -1)
    print("flattened_output: ", flattened_output.shape)


    fc1_output = fully_connected(flattened_output, fc1_wt, fc1_bias)
    print("fully connected layer 1:", fc1_output.shape)

    fc1_output = relu(fc1_output)

    fc2_output = fully_connected(fc1_output, fc2_wt, fc2_bias)
    print("fully connected layer 2:", fc2_output.shape)

    probabilities = softmax(fc2_output)

    predicted_classes = np.argmax(probabilities, axis=1)

    return predicted_classes

In [17]:
pred = forward_pass(test_images[0:100], conv1_wt, conv1_bias, fc1_wt, fc1_bias, fc2_wt, fc2_bias)

print("Predicted values: ", pred[:10])
print("Test Labels     : ", test_labels[:10])
print("accuracy: ", sum(pred[:100] == test_labels[:100])/len(pred[:100]))

input:  (100, 28, 28)
convolution:  (100, 32, 26, 26)
activation:  (100, 32, 26, 26)
max pooling:  (100, 32, 13, 13)
flattened_output:  (100, 5408)
fully connected layer 1: (100, 64)
fully connected layer 2: (100, 10)
Predicted values:  [7 2 1 0 4 1 4 9 6 9]
Test Labels     :  [7 2 1 0 4 1 4 9 5 9]
accuracy:  0.86


In [19]:
def hex_to_float_8bit(hex_value):
    int_value = int(hex_value, 16)
    float_value = int_value / 255.0
    return float_value

In [64]:
def convol_output(flattened_arr):
  hex_val = []
  for i in range(0, len(flattened_arr), 2):
    a = flattened_arr[i]
    b = flattened_arr[i+1]
    hex_val.extend([a+b])

  #print(hex_val)
  conv_out = []
  for i in range(len(hex_val)):
    conv_out.append(hex_to_float_8bit(hex_val[i]))
  print(len(conv_out))
  conv_out = np.array(conv_out)
  conv_out = conv_out.reshape((1, 32, 26, 26))
  return conv_out

In [35]:
flattened_arr = ['0','b','1','d']
convolution_outputs = convol_output(flattened_arr)
convolution_outputs.shape

ValueError: cannot reshape array of size 2 into shape (1,32,28,28)

In [32]:

def forward_pass_simba(input_data,flattened_arr, conv1_wt, conv1_bias, fc1_wt, fc1_bias, fc2_wt, fc2_bias):

    print("input: ", flattened_arr.shape)

    conv_output = convol_output(flattened_arr)
    print("convolution: ", conv_output.shape)

    conv_output = relu(conv_output)
    print("activation: ", conv_output.shape)

    pooled_output = max_pool(conv_output, kernel_size=2)
    print("max pooling: ", pooled_output.shape)

    flattened_output = pooled_output.reshape(len(input_data), -1)
    print("flattened_output: ", flattened_output.shape)


    fc1_output = fully_connected(flattened_output, fc1_wt, fc1_bias)
    print("fully connected layer 1:", fc1_output.shape)

    fc1_output = relu(fc1_output)

    fc2_output = fully_connected(fc1_output, fc2_wt, fc2_bias)
    print("fully connected layer 2:", fc2_output.shape)

    probabilities = softmax(fc2_output)

    predicted_classes = np.argmax(probabilities, axis=1)

    return predicted_classes

In [61]:
flattened_arr = []
for i in range(21632*2):
  flattened_arr.append('01')
flattened_arr = np.array(flattened_arr)
len(flattened_arr)

43264

In [65]:
pred = forward_pass_simba(test_images[0:1], flattened_arr, conv1_wt, conv1_bias, fc1_wt, fc1_bias, fc2_wt, fc2_bias)

print("Predicted values: ", pred[:1])
print("Test Labels     : ", test_labels[:1])
print("accuracy: ", sum(pred[:1] == test_labels[:1])/len(pred[:1]))

input:  (43264,)
21632
convolution:  (1, 32, 26, 26)
activation:  (1, 32, 26, 26)
max pooling:  (1, 32, 13, 13)
flattened_output:  (1, 5408)
fully connected layer 1: (1, 64)
fully connected layer 2: (1, 10)
Predicted values:  [2]
Test Labels     :  [7]
accuracy:  0.0


In [50]:
arr = np.zeros(21632)
print(arr.shape)
arr = arr.reshape((1,32,26,26))
arr.shape

(21632,)


(1, 32, 26, 26)